In [1]:
import praw
import json
import pandas as pd
from pymongo import MongoClient

In [2]:
def create_reddit_object(json_file = "reddit_config.json"):

    with open(json_file) as f:
        data = json.load(f)


    reddit = praw.Reddit(
        user_agent = data['user_agent'],
        client_id = data['client_id'],
        client_secret = data['client_secret'],
        username = data['username'],
        password = data['password']
    )
    
    return reddit

reddit = create_reddit_object()

In [3]:
name_of_subreddit = "news"

In [4]:
subred = reddit.subreddit(name_of_subreddit)

In [5]:
new = subred.hot(limit = 30) # liczba subredditów jakie chcemy zescrapować

Poniżej możemy sprawdzić jakie jeszcze informacje można wyciągnąć z zescrapowanych danych.

In [ ]:
x = next(new)
print(dir(x))

Poniższa komórka pozwala na sprawdzenie "na żywo" co znajduje się w wybranych danych.

In [ ]:
for i in new:
    try:
        content = reddit.submission(url = i.url)
        text = content.selftext
    except: 
        continue
    print(i.title, i.url, len(i.comments), text)

In [90]:
new = subred.hot(limit = 30)

title = []
content = []
noc = []
url = []

for i in new:
    try:    
        text1 = reddit.submission(url = i.url)
        text = text1.selftext
    except:
        continue
    title.append(i.title)
    content.append(text)
    noc.append(len(i.comments))
    url.append(i.url)

data = {
    "Title": title,
    "SubReddit": content,
    "Number of comments": noc,
    "URL": url
}

df = pd.DataFrame(data)

In [ ]:
df

In [92]:
client = MongoClient('localhost', 27017)
db = client['Reddit']
collection = db['SubReddits']

In [93]:
records = df.to_dict(orient = 'records')
collection.insert_many(records)

InsertManyResult([ObjectId('65f3628bb424873c5ea9e903'), ObjectId('65f3628bb424873c5ea9e904')], acknowledged=True)